In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OneHotEncoder 

In [2]:
df = pd.read_csv("D:\machine_learning\Data\Student_Performance.csv")

In [3]:
df.head()

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91
1,4,82,No,4,2,65
2,8,51,Yes,7,2,45
3,5,52,Yes,5,2,36
4,7,75,No,8,5,66


In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Performance Index'],axis=1),df[['Performance Index']],test_size=0.2, random_state=42)

In [8]:
from sklearn.compose import  ColumnTransformer

In [35]:

transformer = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first', sparse_output=False), [2]),
    ('sc',StandardScaler(),[0,1,3,4])
], remainder='passthrough')

In [32]:
ohe = OneHotEncoder(drop='first', sparse_output=False)
ohe.fit_transform(df[['Extracurricular Activities']])

array([[1.],
       [0.],
       [1.],
       ...,
       [1.],
       [1.],
       [0.]])

In [36]:
X_train = transformer.fit_transform(X_train)

X_test = transformer.transform(X_test)

In [40]:
sc = StandardScaler()
y_train= sc.fit_transform(y_train.values.reshape(-1, 1))
y_test = sc.transform(y_test.values.reshape(-1, 1))

In [70]:
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test,dtype=torch.float32)

In [44]:
X_train_tensor.shape

torch.Size([8000, 5])

In [66]:
y_train_tensor.shape

torch.Size([8000, 1])

In [45]:
from torch.utils.data import Dataset,DataLoader 

class CustomDataset(Dataset):
    def __init__(self,features, labels):
        self.features = features 
        self.labels = labels
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self,idx):
        return self.features[idx] , self.labels[idx]

In [71]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)

In [48]:
train_dataset

In [72]:
train_loader = DataLoader(train_dataset,batch_size=100,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=100,shuffle=True)

#### Defining Model

In [50]:
import torch.nn as nn

In [56]:
import torch.optim as optim

In [51]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # input_dim features → 1 output

    def forward(self, x):
        return self.linear(x)

In [54]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

In [92]:
input_dim = X_train_tensor.shape[1]
model = LinearRegressionModel(input_dim)
model.apply(weights_init)
loss_fn = nn.MSELoss()   # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.5,weight_decay=1e-10)


In [93]:
learning_rate = 0.001
epochs = 50

In [94]:
for epoch in range(epochs):
    for batch_features, batch_labels in train_loader:
        y_pred = model(batch_features)
        loss = loss_fn(y_pred,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/50], Loss: 0.0127
Epoch [20/50], Loss: 0.0211
Epoch [30/50], Loss: 0.0244
Epoch [40/50], Loss: 0.0119
Epoch [50/50], Loss: 0.0146


In [96]:
model.eval()
with torch.no_grad():
   y_pred_test = model(X_test_tensor).numpy()
   y_test_true = y_test_tensor.numpy()

In [97]:
from sklearn.metrics import mean_squared_error

In [98]:
y_pred_original = sc.inverse_transform(y_pred_test)
y_test_original = sc.inverse_transform(y_test_true)

# Calculate real MSE
mse = mean_squared_error(y_test_original, y_pred_original)
print(f"\n✅ Final Test MSE (original scale): {mse:.4f}")


✅ Final Test MSE (original scale): 9.2591
